In [95]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
from qreader import QReader

def show_wait_destroy(winname, img):
    cv.imshow(winname, img)
    cv.moveWindow(winname, 500, 0)
    cv.waitKey(0)
    cv.destroyWindow(winname)
def does_range_overlap(startX1,endX1,startX2,endX2):
    if(startX1<=startX2):
        first_range_start=startX1
        first_range_end=endX1
        second_range_start=startX2
        second_range_end=endX2
    else:
        first_range_start=startX2
        first_range_end=endX2
        second_range_start=startX1
        second_range_end=endX1
    return second_range_start <= first_range_end
def midpoint(points):
    sumX=0
    sumY=0
    for point in points:
        sumX=point[0]+sumX
        sumY=point[1]+sumY
    return (sumX/len(points),sumY/len(points))
def findAngle(point1,point2):
    yDiff=abs(point1[1]-point2[1])
    xDiff=abs(point1[0]-point2[0])
    return np.arctan(yDiff/xDiff)
def crop(img,lowerLeftP1,upperRightP2):
    return np.copy(img[int(lowerLeftP1[1]):int(upperRightP2[1]), int(lowerLeftP1[0]):int(upperRightP2[0])])
def getGuagesByQRCodes(img):
    guages=[]
    qreader = QReader()
    qrcodes = qreader.detect(img)
    qrCodeInfos = []
    id = 0
    if len(qrcodes)!=6:
        raise Exception("Less than 6 qr codes detected")
    for qrcode in qrcodes:
        qrcodeCoords = qrcode['quad_xy']
        midPointAprox,lowerLeftPoint,lowerRightPoint,upperLeftPoint,upperRightPoint,isBottom = identifyMatrix(qrcodeCoords)
        qrCodeInfo = {"coords": qrcodeCoords,
                      "id": id, 
                      "lowerLeftPoint": lowerLeftPoint, 
                      "lowerRightPoint": lowerRightPoint, 
                      "upperLeftPoint": upperLeftPoint, 
                      "upperRightPoint": upperRightPoint, 
                      "isBottom": isBottom}
        qrCodeInfos.append(qrCodeInfo)
        id=id+1
    idsAssigned = set()
    for qrCodeInfo in qrCodeInfos:
        if qrCodeInfo["id"] in idsAssigned:
            continue
        idsAssigned.add(qrCodeInfo["id"])
        guage = {}
        if qrCodeInfo["isBottom"]:
            guage["bottomQr"] = qrCodeInfo
        else:
            guage["topQr"] = qrCodeInfo
        for matchingQrCodeInfo in qrCodeInfos:
            if matchingQrCodeInfo["id"] in idsAssigned:
                continue
            if does_range_overlap(matchingQrCodeInfo["lowerLeftPoint"][0],
                                  matchingQrCodeInfo["lowerRightPoint"][0],
                                  qrCodeInfo["lowerLeftPoint"][0],
                                  qrCodeInfo["lowerRightPoint"][0]):
                if matchingQrCodeInfo["isBottom"]:
                    guage["bottomQr"] = matchingQrCodeInfo
                else:
                    guage["topQr"] = matchingQrCodeInfo
                idsAssigned.add(matchingQrCodeInfo["id"])
                break
        if "topQr" not in guage or "bottomQr" not in guage:
            raise Exception(f"One of the guges {guage} from assigned {idsAssigned} did not have a bottom or top QR: {qrCodeInfos}")
        guages.append(guage)
    return guages
def identifyMatrix(qrPoints):
    midPointAprox=midpoint(qrPoints)
    lowerLeftPoint=None
    upperRightPoint=None
    lowerRightPoint=None
    upperLeftPoint=None
    isBottom=qrPoints[0][1]>500
    for qrPoint in qrPoints:
        if(qrPoint[0]>midPointAprox[0]):
            #right
            if qrPoint[1]<midPointAprox[1]:
                #top
                upperRightPoint=qrPoint
            else:
                #bottom
                lowerRightPoint=qrPoint
        else:
            #left
            if qrPoint[1]<midPointAprox[1]:
                #top
                upperLeftPoint=qrPoint
            else:
                #bottom
                lowerLeftPoint=qrPoint
    return midPointAprox,lowerLeftPoint,lowerRightPoint,upperLeftPoint,upperRightPoint,isBottom
def getGuageTracks(guages):
    guageTracks = []
    for guage in guages:
        guageTracks.append({"ids":[guage["topQr"]["id"],guage["bottomQr"]["id"]], 
                            "topPoint":(int((guage["topQr"]["upperLeftPoint"][0]+guage["topQr"]["upperRightPoint"][0])/2),int(max(guage["topQr"]["upperLeftPoint"][1],guage["topQr"]["upperRightPoint"][1]))),
                            "bottomPoint":(int((guage["bottomQr"]["lowerLeftPoint"][0]+guage["bottomQr"]["lowerRightPoint"][0])/2),int(max(guage["bottomQr"]["lowerLeftPoint"][1],guage["bottomQr"]["lowerRightPoint"][1]))),
                            "yTop":max(guage["topQr"]["lowerLeftPoint"][1],guage["topQr"]["lowerRightPoint"][1])+10,
                            "yBottom":min(guage["bottomQr"]["upperLeftPoint"][1],guage["bottomQr"]["upperRightPoint"][1])-10
                           })
    return guageTracks
def indicatorSizeUsingEdge(startX,endX,yMid,edges):
    startIdx=-1
    endIdx=-1
    indicatorSize=0
    xCoords=startX
    while xCoords <endX and xCoords<len(edges[0]):
        if edges[int(yMid)][int(xCoords)]!=0:
            if startIdx == -1:
                startIdx = xCoords
            else:
                endIdx = xCoords
        xCoords=xCoords+1
        if startIdx == -1 or endIdx == -1 :
            indicatorSize = 0
        else:
            indicatorSize = endIdx - startIdx
    return indicatorSize
def indicatorSizeUsingContinuous(startX,endX,yMid,bit_not):
    startIdx=-1
    endIdx=-1
    indicatorSize=0
    xCoords=startX
    while xCoords <endX and xCoords<len(bit_not[0]):
        if bit_not[int(yMid)][int(xCoords)]!=0:
            if startIdx == -1:
                startIdx = xCoords
            else:
                endIdx = xCoords
        else:
            if endIdx-startIdx > indicatorSize:
                indicatorSize = endIdx-startIdx
            startIdx=-1
            endIdx=-1
        xCoords=xCoords+1
    return indicatorSize
def indicatorSizeUsingVerticalDistance(startX,endX,yMid,testHeight,bit_not):
    startIdx=-1
    endIdx=-1
    indicatorSize=0
    xCoords=startX
    while xCoords <endX and xCoords<len(bit_not[0]):
        if bit_not[int(yMid)][int(xCoords)]!=0:
            height = yMid
            while height < yMid + testHeight:
                if bit_not[int(height)][int(xCoords)]==0:
                    break
                height += 1
            if height-yMid >indicatorSize:
                indicatorSize = height-yMid
        xCoords=xCoords+1
    return indicatorSize
def getXCoord(yCoord,slope,intercept):
    return (yCoord - intercept)/slope

In [114]:
img = cv.imread('calibration/undistorted/ch1-1765804312.jpg')
mtx=np.loadtxt('cameraMatrix.array')
dist=np.loadtxt('distanceCoefficients.array')
rvecs=np.loadtxt('rotation.array').reshape(1,3,1)
tvecs=np.loadtxt('translation.array').reshape(1,3,1)
h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
img = cv.undistort(img, mtx, dist, None, newcameramtx)

In [123]:
show_wait_destroy("img",img)


error: OpenCV(4.13.0) /io/opencv/modules/highgui/src/window_QT.cpp:578: error: (-27:Null pointer) NULL guiReceiver (please create a window) in function 'cvDestroyWindow'


In [ ]:
img = cv.imread('calibration/undistorted/ch0-1765830911.jpg')
img = cv.imread('calibration/undistorted/ch0-1765835383.jpg')
img = cv.imread('calibration/undistorted/ch0-1765922689.jpg')
img = cv.imread('calibration/undistorted/ch0-1765923535.jpg')
img = cv.imread('calibration/undistorted/ch0-1765928533.jpg')
img = cv.imread('calibration/undistorted/ch0-1765928636.jpg')
img = cv.imread('calibration/undistorted/ch0-1765928811.jpg')
img = cv.imread('calibration/undistorted/ch0-1765928888.jpg')
img = cv.imread('calibration/undistorted/ch0-1765933150.jpg')
img = cv.imread('calibration/undistorted/ch0-1765973813.jpg')
img = cv.imread('calibration/undistorted/ch0-1765974884.jpg')
img = cv.imread('calibration/undistorted/ch0-1765513908.jpg')



aproxIndicatorLength=600
indicatorBuffer=40
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
blur = cv.GaussianBlur(img, (11,11), 0)
hsl = cv.cvtColor(blur, cv.COLOR_BGR2HLS)
lower_silver = np.array([0,0,200])
upper_silver = np.array([255,255,255])
mask = cv.inRange(hsv, lower_silver, upper_silver)
#show_wait_destroy("blur",blur)
#show_wait_destroy("mask",mask)

guages = getGuagesByQRCodes(img)
guageTracks = getGuageTracks(guages)
highlightImg = np.copy(img)

for guage in guages:
    bottomQR = guage["bottomQr"]
    cv.circle(highlightImg,(int(bottomQR["lowerLeftPoint"][0]),int(bottomQR["lowerLeftPoint"][1])), 6, (255,0,0), -1)
    cv.circle(highlightImg,(int(bottomQR["lowerRightPoint"][0]),int(bottomQR["lowerRightPoint"][1])), 6, (255,0,0), -1)
    cv.circle(highlightImg,(int(bottomQR["upperLeftPoint"][0]),int(bottomQR["upperLeftPoint"][1])), 6, (255,0,0), -1)
    cv.circle(highlightImg,(int(bottomQR["upperRightPoint"][0]),int(bottomQR["upperRightPoint"][1])), 6, (255,0,0), -1)
    topQR = guage["topQr"]
    cv.circle(highlightImg,(int(topQR["lowerLeftPoint"][0]),int(topQR["lowerLeftPoint"][1])), 6, (255,0,0), -1)
    cv.circle(highlightImg,(int(topQR["lowerRightPoint"][0]),int(topQR["lowerRightPoint"][1])), 6, (255,0,0), -1)
    cv.circle(highlightImg,(int(topQR["upperLeftPoint"][0]),int(topQR["upperLeftPoint"][1])), 6, (255,0,0), -1)
    cv.circle(highlightImg,(int(topQR["upperRightPoint"][0]),int(topQR["upperRightPoint"][1])), 6, (255,0,0), -1)

for guageTrack in guageTracks:
    coefficients = np.polyfit([guageTrack["bottomPoint"][0],guageTrack["topPoint"][0]],
                              [guageTrack["bottomPoint"][1],guageTrack["topPoint"][1]],1)
    slope, intercept = coefficients
    yMid = guageTrack["yTop"]
    yMax = guageTrack["yBottom"]
    guageIndicatorSizes = []
    averageIndicatorSum = 0
    averageIndicatorCount = 0
    while yMid<guageTrack["bottomPoint"][1]+aproxIndicatorLength and yMid < yMax:
        xMid = getXCoord(yMid,slope,intercept)
        xCoords = xMid-30
        indicatorSize = indicatorSizeUsingVerticalDistance(xCoords,xMid +30,yMid,50,mask)
        if indicatorSize > 0:
            guageIndicatorSizes.append((int(xMid+50),int(yMid),indicatorSize))
            averageIndicatorSum=averageIndicatorSum+indicatorSize
            averageIndicatorCount=averageIndicatorCount+1
        yMid = yMid + 1
    averageIndicatorAverage = averageIndicatorSum/averageIndicatorCount
    indicatedPixel=0
    indicatedPixelX=0
    indicatedWidth=0
    for guageIndicatorSize in guageIndicatorSizes:
        if guageIndicatorSize[2]>averageIndicatorAverage:
            highlightImg=cv.circle(highlightImg,(int(guageIndicatorSize[0]+50),int(guageIndicatorSize[1])), min(int(guageIndicatorSize[2]/7),5), (0,0,255), -1)
            if guageIndicatorSize[2] > indicatedWidth:
                indicatedPixel=guageIndicatorSize[1]
                indicatedPixelX=guageIndicatorSize[0]
                indicatedWidth=guageIndicatorSize[2]
            print(f"x= {xMid} y={yMid} {guageIndicatorSize[2]}")
    percent=(((indicatedPixel-guageTrack["yBottom"])*-100)/(guageTrack["yBottom"]-guageTrack["yTop"]))
    highlightImg=cv.line(highlightImg,(indicatedPixelX,indicatedPixel),(indicatedPixelX+20,indicatedPixel),(255,0,0),2)
    highlightImg=cv.putText(highlightImg,str(f"{percent}%"),(indicatedPixelX,indicatedPixel), cv.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2,cv.LINE_AA)
    highlightImg=cv.putText(highlightImg,str(f"0%"),(int(getXCoord(guageTrack["yBottom"],slope,intercept)),int(guageTrack["yBottom"])), cv.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2,cv.LINE_AA)
    highlightImg=cv.putText(highlightImg,str(f"100%"),(int(getXCoord(guageTrack["yTop"],slope,intercept)),int(guageTrack["yTop"])), cv.FONT_HERSHEY_SIMPLEX, 1,(0,255,255),2,cv.LINE_AA)
    highlightImg=cv.line(highlightImg,(guageTrack["topPoint"][0],guageTrack["topPoint"][1]),(guageTrack["bottomPoint"][0],guageTrack["bottomPoint"][1]),(255,0,0),2)
#show_wait_destroy("highlightImg",highlightImg)

x= 555.6177572048846 y=778.2659912109375 19.0
x= 555.6177572048846 y=778.2659912109375 18.0
x= 555.6177572048846 y=778.2659912109375 23.0
x= 555.6177572048846 y=778.2659912109375 22.0
x= 555.6177572048846 y=778.2659912109375 21.0
x= 555.6177572048846 y=778.2659912109375 20.0
x= 555.6177572048846 y=778.2659912109375 19.0
x= 555.6177572048846 y=778.2659912109375 18.0
x= 555.6177572048846 y=778.2659912109375 17.0
x= 555.6177572048846 y=778.2659912109375 16.0
x= 555.6177572048846 y=778.2659912109375 15.0
x= 555.6177572048846 y=778.2659912109375 14.0
x= 555.6177572048846 y=778.2659912109375 13.0
x= 555.6177572048846 y=778.2659912109375 12.0
x= 555.6177572048846 y=778.2659912109375 12.0
x= 555.6177572048846 y=778.2659912109375 26.0
x= 555.6177572048846 y=778.2659912109375 36.0
x= 555.6177572048846 y=778.2659912109375 35.0
x= 555.6177572048846 y=778.2659912109375 34.0
x= 555.6177572048846 y=778.2659912109375 33.0
x= 555.6177572048846 y=778.2659912109375 32.0
x= 555.6177572048846 y=778.2659912

In [7]:
img = cv.imread('calibration/undistorted/ch0-1765830911.jpg')
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)

aproxIndicatorLength=600
indicatorBuffer=40
qr = cv.QRCodeDetector()
ret_qr, codes = qr.detectMulti(img)
guage_ranges = set()
possibleGuages = []
highlightImg = np.copy(img)
qrInfo={}
idx=0
bit_not = cv.adaptiveThreshold(cv.cvtColor(img, cv.COLOR_BGR2GRAY),255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 19, -2)
if ret_qr:
    for points in codes:
        startX = min(points[0][0],points[1][0],points[2][0],points[3][0])
        endX = max(points[0][0],points[1][0],points[2][0],points[3][0])
        guage_ranges.add((startX,endX,idx))
        qrInfo[idx]={"points":points}
        idx=idx+1
    unique_ranges=[]
    remaining_ranges=set(guage_ranges)
    for guage_range1 in guage_ranges:
        range_to_add = None
        guages_to_remove = set()
        if guage_range1 in remaining_ranges:
            range_to_add = {"rangeX": guage_range1, "ranges": [guage_range1]}
            remaining_ranges.remove(guage_range1)
        for guage_range2 in remaining_ranges:
            if(does_range_overlap(guage_range1[0],guage_range1[1],guage_range2[0],guage_range2[1])):
                range_to_add["ranges"].append(guage_range2)
                range_to_add["rangeX"]=(min(range_to_add["rangeX"][0],guage_range2[0]),max(range_to_add["rangeX"][1],guage_range2[1]))
                guages_to_remove.add(guage_range2)
        if range_to_add:
            remaining_ranges=remaining_ranges-guages_to_remove
            minY=100000
            maxY=0
            theta=0
            if len(range_to_add["ranges"])>1:
                for range_info in range_to_add["ranges"]:
                    qrPoints=qrInfo[range_info[2]]["points"]
                    minY=min(minY,qrPoints[0][1],qrPoints[1][1],qrPoints[2][1],qrPoints[3][1])
                    maxY=max(maxY,qrPoints[0][1],qrPoints[1][1],qrPoints[2][1],qrPoints[3][1])
                    midPointAprox,lowerLeftPoint,lowerRightPoint,upperLeftPoint,upperRightPoint,isBottom = identifyMatrix(qrPoints)
                    highlightImg=cv.putText(highlightImg,str(range_info[2]),(int(midPointAprox[0]),int(midPointAprox[1])), cv.FONT_HERSHEY_SIMPLEX, 4,(255,255,255),2,cv.LINE_AA)
                    highlightImg=cv.circle(highlightImg,(int(lowerLeftPoint[0]),int(lowerLeftPoint[1])), 7, (0,0,255), -1)
                    highlightImg=cv.circle(highlightImg,(int(upperRightPoint[0]),int(upperRightPoint[1])), 5, (0,255,0), -1)
                    highlightImg=cv.circle(highlightImg,(int(lowerRightPoint[0]),int(lowerRightPoint[1])), 5, (255,0,0), -1)
                    highlightImg=cv.circle(highlightImg,(int(upperLeftPoint[0]),int(upperLeftPoint[1])), 5, (255,0,0), -1)
                    theta=findAngle(lowerLeftPoint,lowerRightPoint)
                    if lowerRightPoint[1]<lowerLeftPoint[1]:
                        theta = theta*-1
                    print(f"Angle {str(range_info[2])} {theta*(180/np.pi)}")
                    if isBottom :
                        startX=lowerLeftPoint[0]+((lowerRightPoint[0]-lowerLeftPoint[0])/2)
                        startY=max(lowerLeftPoint[1],lowerRightPoint[1])
                        a = np.tan(theta-(np.pi/2))
                        b = startY - (a*startX)
                        endY=startY-aproxIndicatorLength
                        endX=(endY-b)/a
                        highlightImg=cv.line(highlightImg,(int(startX),int(startY)),(int(endX),int(endY)),(255,0,0),2)
                        highlightImg=cv.line(highlightImg,(int(startX)-indicatorBuffer,int(startY)),(int(endX)-indicatorBuffer,int(endY)),(255,0,0),2)
                        highlightImg=cv.line(highlightImg,(int(startX)+indicatorBuffer,int(startY)),(int(endX)+indicatorBuffer,int(endY)),(255,0,0),2)
                        range_to_add["bottom"]=qrPoints
                        range_to_add["startLine"]=(int(startX),int(startY))
                    else:
                        startX=upperLeftPoint[0]+((upperRightPoint[0]-upperLeftPoint[0])/2)
                        startY=max(upperLeftPoint[1],upperRightPoint[1])
                        a = np.tan((-1*theta)-(np.pi/2))
                        b = startY - (a*startX)
                        endY=startY+aproxIndicatorLength
                        endX=(endY-b)/a
                        highlightImg=cv.line(highlightImg,(int(startX),int(startY)),(int(endX),int(endY)),(0,255,0),2)
                        highlightImg=cv.line(highlightImg,(int(startX)-indicatorBuffer,int(startY)),(int(endX)-indicatorBuffer,int(endY)),(0,255,0),2)
                        highlightImg=cv.line(highlightImg,(int(startX)+indicatorBuffer,int(startY)),(int(endX)+indicatorBuffer,int(endY)),(0,255,0),2)
                        range_to_add["top"]=qrPoints
                        range_to_add["endLine"]=(int(startX),int(startY))
                highlightImg=cv.line(highlightImg,range_to_add["startLine"],range_to_add["endLine"],(0,0,255),2)
            else:
                qrPoints = qrInfo[range_to_add["ranges"][0][2]]["points"]
                if(qrPoints[0][1]>500):
                    #top half
                    minY=0
                    maxY=max(maxY,qrPoints[0][1],qrPoints[1][1],qrPoints[2][1],qrPoints[3][1])
                else:
                    #bottom half
                    minY=min(minY,qrPoints[0][1],qrPoints[1][1],qrPoints[2][1],qrPoints[3][1])
                    maxY=1072
                midPointAprox,lowerLeftPoint,lowerRightPoint,upperLeftPoint,upperRightPoint,isBottom = identifyMatrix(qrPoints)
                highlightImg=cv.putText(highlightImg,str(range_to_add["ranges"][0][2]),(int(midPointAprox[0]),int(midPointAprox[1])), cv.FONT_HERSHEY_SIMPLEX, 4,(255,255,255),2,cv.LINE_AA)
                highlightImg=cv.circle(highlightImg,(int(lowerLeftPoint[0]),int(lowerLeftPoint[1])), 7, (0,0,255), -1)
                highlightImg=cv.circle(highlightImg,(int(upperRightPoint[0]),int(upperRightPoint[1])), 5, (0,255,0), -1)
                highlightImg=cv.circle(highlightImg,(int(lowerRightPoint[0]),int(lowerRightPoint[1])), 5, (255,0,0), -1)
                highlightImg=cv.circle(highlightImg,(int(upperLeftPoint[0]),int(upperLeftPoint[1])), 5, (255,0,0), -1)
                theta=findAngle(lowerLeftPoint,lowerRightPoint)
                if lowerRightPoint[1]<lowerLeftPoint[1]:
                    theta = theta*-1
                print(f"Angle {str(range_to_add["ranges"][0][2])} {theta*(180/np.pi)}")
                if isBottom :
                    startX=lowerLeftPoint[0]+((lowerRightPoint[0]-lowerLeftPoint[0])/2)
                    startY=max(lowerLeftPoint[1],lowerRightPoint[1])
                    a = np.tan(theta-(np.pi/2))
                    b = startY - (a*startX)
                    endY=startY-aproxIndicatorLength
                    endX=(endY-b)/a
                    highlightImg=cv.line(highlightImg,(int(startX),int(startY)),(int(endX),int(endY)),(255,0,0),2)
                    highlightImg=cv.line(highlightImg,(int(startX)-indicatorBuffer,int(startY)),(int(endX)-indicatorBuffer,int(endY)),(255,0,0),2)
                    highlightImg=cv.line(highlightImg,(int(startX)+indicatorBuffer,int(startY)),(int(endX)+indicatorBuffer,int(endY)),(255,0,0),2)
                    range_to_add["bottom"]=qrPoints
                    range_to_add["startLine"]=(int(startX),int(startY))
                    range_to_add["endLine"]=(int(endX),int(endY))
                else:
                    startX=upperLeftPoint[0]+((upperRightPoint[0]-upperLeftPoint[0])/2)
                    startY=max(upperLeftPoint[1],upperRightPoint[1])
                    a = np.tan((-1*theta)-(np.pi/2))
                    b = startY - (a*startX)
                    endY=startY+aproxIndicatorLength
                    endX=(endY-b)/a
                    highlightImg=cv.line(highlightImg,(int(startX),int(startY)),(int(endX),int(endY)),(0,255,0),2)
                    highlightImg=cv.line(highlightImg,(int(startX)-indicatorBuffer,int(startY)),(int(endX)-indicatorBuffer,int(endY)),(0,255,0),2)
                    highlightImg=cv.line(highlightImg,(int(startX)+indicatorBuffer,int(startY)),(int(endX)+indicatorBuffer,int(endY)),(0,255,0),2)
                    range_to_add["top"]=qrPoints
                    range_to_add["startLine"]=(int(startX),int(startY))
                    range_to_add["endLine"]=(int(endX),int(endY))
            range_to_add["minY"]=minY
            range_to_add["maxY"]=maxY
            print((int(range_to_add["rangeX"][0]),int(minY)))
            print((int(range_to_add["rangeX"][1]),int(maxY)))
            highlightImg=cv.rectangle(highlightImg,(int(range_to_add["rangeX"][0]),int(minY)),(int(range_to_add["rangeX"][1]),int(maxY)),(255,0,0),2)
            possibleGuage=crop(img,(int(range_to_add["rangeX"][0]),int(minY)),(int(range_to_add["rangeX"][1]),int(maxY)))
            range_to_add["possibleGuage"] = possibleGuage
            unique_ranges.append(range_to_add)
            coefficients = np.polyfit([range_to_add["startLine"][0],range_to_add["endLine"][0]],[range_to_add["startLine"][1],range_to_add["endLine"][1]],1)
            slope, intercept = coefficients
            yMid = range_to_add["startLine"][1]
            if range_to_add["endLine"][1] < yMid :
                yMid = range_to_add["endLine"][1]
            yMax = max(range_to_add["endLine"][1],range_to_add["startLine"][1])
            print(f"startLine: {range_to_add["startLine"]} endLine:{range_to_add["endLine"]} slope: {slope} intercept: {intercept}")
            while yMid<range_to_add["startLine"][1]+aproxIndicatorLength and yMid < yMax:
                xMid = (yMid - intercept)/slope
                xCoords = xMid-30
                matchCount=0
                while xCoords <xMid +30 and xCoords<len(bit_not):
                    if bit_not[int(xCoords)][int(yMid)]!=0:
                        matchCount=matchCount+1
                    xCoords=xCoords+1
                if matchCount > 50:
                    highlightImg=cv.circle(highlightImg,(int(xMid),int(yMid)), 7, (0,0,255), -1)
                print(f"x= {xMid} y={yMid} {matchCount}")
                yMid = yMid + 1
            
show_wait_destroy("highlightImg",highlightImg)
show_wait_destroy("bit_not",bit_not)


Angle 1 25.947093963623047
(210, 0)
(471, 1057)
startLine: (340, 983) endLine:(632, 383) slope: -2.0547945205479445 intercept: 1681.6301369863013
x= 632.0000000000001 y=383 15
x= 631.5133333333335 y=384 31
x= 631.0266666666669 y=385 33
x= 630.5400000000002 y=386 38
x= 630.0533333333335 y=387 53
x= 629.5666666666668 y=388 48
x= 629.0800000000002 y=389 36
x= 628.5933333333335 y=390 20
x= 628.1066666666668 y=391 11
x= 627.6200000000001 y=392 1
x= 627.1333333333336 y=393 0
x= 626.6466666666669 y=394 0
x= 626.1600000000002 y=395 0
x= 625.6733333333335 y=396 0
x= 625.1866666666668 y=397 0
x= 624.7000000000002 y=398 0
x= 624.2133333333335 y=399 0
x= 623.7266666666668 y=400 0
x= 623.2400000000001 y=401 4
x= 622.7533333333334 y=402 5
x= 622.2666666666669 y=403 35
x= 621.7800000000002 y=404 50
x= 621.2933333333335 y=405 46
x= 620.8066666666668 y=406 37
x= 620.3200000000002 y=407 32
x= 619.8333333333335 y=408 36
x= 619.3466666666668 y=409 35
x= 618.8600000000001 y=410 44
x= 618.3733333333334 y=41